# 8 Queens
## Abraham Maximiliano Ávalos Corrales
## David Omar Paredes Paredes

In [1]:
import numpy  as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LinearRegression
from sklearn.metrics         import mean_absolute_error
import matplotlib.pyplot as plt
import random

In [2]:
class Problem:
	def generate_initial_population(self, p_size) -> list:...
	def _compare(self, fitness_a, fitness_b) -> int:...

	def get_parent(self, parent_F:list) -> int:...
	def mix_parents(self, parent_a, parent_b, ratio:float=0.5):...
	def mutate(self, g):...
	def check_individual_integrity(self, g) -> bool: return True

	def get_fitness(self, G:list, F:list=None):...
	def get_elite(self, G:list, F:list) -> (list, list):...
	def deep_copy(self, G, population:bool=True):...
	def update_elite(self, E, newE, Fe, newFe):...

	def solved(self, Fe:list): return False
	def has_progress_metric(self) -> bool: return False
	def get_progress_metric(self, F:list) -> float:...
	def get_progress_max(self) -> float:...
	def get_variation(self, F:list) -> float:...
	def custom_print(self, G:list, F:list):...

In [3]:
class Genetic_solver:
	def __init__(self
		,problem:Problem
		,p_size:int       = 50
		,max_it:int       = 1000
		,pCross:float     = 0.5
		,pMut:float       = 0.5
		,rng              = None
		# Formatting options for progress bar
		,FEEDBACK_INT:int = 10
		,MAX_F_LEN:int    = 30
		,MAX_K_LEN:int    = 20
		,SPINNER:list[str]= ['\\', '|', '/', '-']
		,verbose:bool     = False
		,use_custom_print:bool= False
	):
		self.problem      = problem
		self.p_size       = p_size
		self.max_it       = max_it
		self.pCross       = pCross
		self.pMut         = pMut
		self.rng          = rng
		if rng is None:
			self.rng = np.random.default_rng()
		# Formatting options for progress bar
		self.FEEDBACK_INT = FEEDBACK_INT
		self.MAX_F_LEN    = MAX_F_LEN
		self.MAX_K_LEN    = MAX_K_LEN
		self.SPINNER      = SPINNER
		self.SPINNER_LEN  = len(self.SPINNER)
		self.verbose      = verbose
		self.use_custom_print= use_custom_print

		# solve() variables
		self.G       = None
		self.F       = None
		self.newG    = None
		self.newF    = None
		self.elite   = None
		self.Fe      = None
		self.invalid = 0
		self.n_it    = 0
		self.converged:bool = False
	
	def _print_progress(self, n_k:int):
		if self.verbose:
			print(f'\nIteration: {n_k}/{self.max_it}')
			if self.use_custom_print:
				self.problem.custom_print(self.G, self.Fe)
			else:
				print(f'///Fitness///')
				print(np.array(self.F))
				print(f'///Elite///')
				print(np.array(self.Fe))
		else:
			spinner_idx:int = int(n_k/self.FEEDBACK_INT)%self.SPINNER_LEN
			k_percentage    = n_k/float(self.max_it)*100
			filled_k        = int(self.MAX_K_LEN *n_k // self.max_it)
			k_bar           = '#'*filled_k + '-'*(self.MAX_K_LEN - filled_k)
			
			print(f'\r{self.SPINNER[spinner_idx]}', end='')
			print(f' Iteration:[{k_bar}]{k_percentage:.2f}%', end=' ')

			if self.problem.has_progress_metric():
				progress = self.problem.get_progress_metric(self.Fe)
				filled_p = int(self.MAX_F_LEN * progress // self.problem.get_progress_max())
				p_bar = '#'*filled_p + '-'*(self.MAX_F_LEN - filled_p)
				variation = self.problem.get_variation(self.F)

				print(f'Fitness:[{p_bar}]pm {variation:.2f}', end='')
			#----------
	# end _print_progress

	def solve(self):
		self.G    = self.problem.generate_initial_population(self.p_size)
		self.newG = [None for i in range(self.p_size)]
		self.F    = self.problem.get_fitness(self.G)

		self.elite, self.Fe = self.problem.get_elite(self.G, self.F)

		for k in range(1, self.max_it +1):
			self.n_it = k
			if self.problem.solved(self.Fe):
				self._print_progress(k)
				self.converged = True
				break

			for i, gntype in enumerate(self.G):
				gntype_cpy = self.problem.deep_copy(gntype, population=False)
				if self.rng.random() < self.pCross:
					parenta_idx  = self.problem.get_parent(self.F)
					parentb_idx  = self.problem.get_parent(self.F)
					self.newG[i] = self.problem.mix_parents(self.G[parenta_idx], self.G[parentb_idx])
				else:
					self.newG[i] = self.problem.deep_copy(gntype, population=False)

				if self.rng.random() < self.pMut:
					self.newG[i] = self.problem.mutate(gntype)
				if not self.problem.check_individual_integrity(self.newG[i]):
					self.invalid += 1
					self.newG[i] = gntype_cpy
			#----------- end for
			Fi = self.problem.get_fitness(self.newG)

			elite_i   , Fe_i    = self.problem.get_elite(self.newG, Fi)
			self.elite, self.Fe = self.problem.update_elite(self.elite, elite_i, self.Fe, Fe_i)

			self.G = self.problem.deep_copy(self.newG)
			self.F = np.array(Fi)

			if k%self.FEEDBACK_INT == 1 or k==self.max_it:
				self._print_progress(k)
		#------------- end for
		print(' Ok!')
		return self.elite
	#---------- end solve